In [2]:
#데이터 처리
import pandas as pd
import numpy as np
import datetime

#시각화
import matplotlib.pyplot as plt
import seaborn as sns

#모델링
from sklearn.ensemble import RandomForestRegressor

#사용자 정의 함수
import interpolation

##warning 메시지 숨기기
import warnings
warnings.filterwarnings('ignore')

In [7]:
#사용한 데이터 불러오기
data_raw=pd.read_csv('funda_train.csv')
#ju=pd.read_csv('memorial.csv',header=None)
#temp=pd.read_csv('focast_temp.csv')
#rain=pd.read_csv('focast_rain.csv')
print(data_raw)

         store_id  card_id card_company transacted_date transacted_time  \
0               0        0            b      2016-06-01           13:13   
1               0        1            h      2016-06-01           18:12   
2               0        2            c      2016-06-01           18:52   
3               0        3            a      2016-06-01           20:22   
4               0        4            c      2016-06-02           11:06   
...           ...      ...          ...             ...             ...   
6556608      2136  4663855            d      2019-02-28           23:20   
6556609      2136  4663855            d      2019-02-28           23:24   
6556610      2136  4663489            a      2019-02-28           23:24   
6556611      2136  4663856            d      2019-02-28           23:27   
6556612      2136  4658616            c      2019-02-28           23:54   

         installment_term  region type_of_business       amount  
0                       0     NaN

### 2. 데이터 전처리
### Data Cleansing & Pre-Processing

In [17]:
## 전처리를 쉽게 하기 위한 (가)변수 추가 
data_raw['action']=1 # 매 결재를 1로 취급하여 일별/ 월별 결재횟수 카운트 
data_raw['year_month']=[i[:7] for i in data_raw.transacted_date] # 년월 변수로 년-월별 매출을 group sum 계산 

## 전처리 편의를 위해 np.na를 문자 'Nan'로 변경
data_raw["type_of_business"].fillna("Nan", inplace = True)
data_raw["region"].fillna("Nan", inplace = True)
print(data_raw)

         store_id  card_id card_company transacted_date transacted_time  \
0               0        0            b      2016-06-01           13:13   
1               0        1            h      2016-06-01           18:12   
2               0        2            c      2016-06-01           18:52   
3               0        3            a      2016-06-01           20:22   
4               0        4            c      2016-06-02           11:06   
...           ...      ...          ...             ...             ...   
6556608      2136  4663855            d      2019-02-28           23:20   
6556609      2136  4663855            d      2019-02-28           23:24   
6556610      2136  4663489            a      2019-02-28           23:24   
6556611      2136  4663856            d      2019-02-28           23:27   
6556612      2136  4658616            c      2019-02-28           23:54   

         installment_term  region type_of_business       amount  action  \
0                       

In [19]:
#스토어별로 년-월별 매출 합 계산
main_data_buff=data_raw[['year_month','store_id','amount']].groupby(['store_id','year_month']).sum()
print(main_data_buff)

                           amount
store_id year_month              
0        2016-06     7.470000e+05
         2016-07     1.005000e+06
         2016-08     8.715714e+05
         2016-09     8.978571e+05
         2016-10     8.354286e+05
...                           ...
2136     2018-10     2.012214e+06
         2018-11     2.135429e+06
         2018-12     2.427429e+06
         2019-01     1.873643e+06
         2019-02     2.227429e+06

[60232 rows x 1 columns]


In [22]:
#매출이 없는 달에 0값 삽임
#최초 매출이 발생하기 전 시점도 동일하게 0으로 취급
main_data_buff.reset_index(level=[0,1], inplace=True)
main_data_buff=interpolation.interpolation(main_data_buff,method='zero') #r
main_data=main_data_buff
print(main_data_buff)

IndexError: Too many levels: Index has only 1 level, not 2

In [23]:
# 매출이 없는 달에 0값 삽입
# 최초 매출이 발생하기 전 시점도 동일하게 0으로 취급
main_data_buff.reset_index(level=[0,1], inplace=True)
main_data_buff=interpolation.interpolation(main_data_buff, method='zero') #r
main_data=main_data_buff

IndexError: Too many levels: Index has only 1 level, not 2

In [ ]:
#year_month_id 추가
ymi=np.concatenate((np.unique(main_data['year_month'].values)))